In [4]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from scipy import stats
from sklearn import linear_model
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import tree
from sklearn import ensemble
from sklearn import metrics
from sklearn import cluster
from sklearn import feature_selection

In [6]:
from google.colab import drive

ModuleNotFoundError: No module named 'google.colab'

In [5]:
# get size of the variable in megabytes
def get_variable_size(var):
    display(sys.getsizeof(var)/ (1024**2))

In [6]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
train_data = pd.read_csv('/content/drive/MyDrive/UrfuCsvDatasets/RegressionProject5SecondSemester/train_data_4.csv')

In [8]:
X = train_data.drop(['trip_duration', 'trip_duration_log', 'visibility', 'wind speed', 'precip'], axis=1)
y = train_data['trip_duration']
y_log = train_data['trip_duration_log']

In [9]:
X_train, X_valid, y_train_log, y_valid_log = model_selection.train_test_split(
    X,
    y_log,
    test_size=0.33,
    random_state=42
)

## 5.

### 5.1.

In [10]:
model = linear_model.LinearRegression()
model.fit(X_train, y_train_log)

LinearRegression()

In [11]:
y_train_predict = model.predict(X_train)
display(y_train_predict)

array([6.26135326, 5.84616734, 5.65496566, ..., 6.12920585, 5.5888809 ,
       6.24565967])

In [12]:
y_valid_predict = model.predict(X_valid)
display(y_valid_predict)

array([6.00913683, 5.69848861, 6.26469989, ..., 6.39176013, 6.68692993,
       6.27534049])

In [13]:
def calculate_rmsle(y_true, y_pred):
    return round(metrics.mean_squared_error(y_true, y_pred, squared=False), 2)

In [16]:
calculate_rmsle(y_train_log, y_train_predict)

0.42

In [15]:
calculate_rmsle(y_valid_log, y_valid_predict)

0.42

### 5.2.

In [ ]:
poly_features = preprocessing.PolynomialFeatures(degree=2, include_bias=False)

X_train_poly = poly_features.fit_transform(X_train)
X_valid_poly = poly_features.fit_transform(X_valid)

linear_reg_poly = linear_model.LinearRegression()
linear_reg_poly.fit(X_train_poly, y_train_log)

y_train_poly_predict = linear_reg_poly.predict(X_train_poly)
print(f"y_train_poly_predict: {y_train_poly_predict}")
y_valid_poly_predict = linear_reg_poly.predict(X_valid_poly)
print(f"y_valid_poly_predict: {y_valid_poly_predict}")

print("RMSLE train_poly:", calculate_rmsle(y_train_log, y_train_poly_predict))
print("RMSLE train:", calculate_rmsle(y_valid_log, y_valid_poly_predict))

### 5.3.